In [1]:
#Inital Imports
import numpy as np
import networkx as nx
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

## Step 1: Parse Network Topology and load Demands

In [2]:
# load data fram using pandas for topology
df = pd.read_csv("data/ATT/topology.txt", delimiter=r"\s+")
print(df)

# Create Graph

G = nx.from_pandas_edgelist(df, 'from_node', 'to_node', ['capacity', 'prob_failure'])

     to_node  from_node  capacity  prob_failure
0          9          2   1200000         0.001
1          9         22   1200000         0.001
2          9          8   1200000         0.001
3          9         13   1200000         0.001
4          9          3   1200000         0.001
..       ...        ...       ...           ...
107       21         17   1200000         0.001
108       21         15   1200000         0.001
109       24         22   1200000         0.001
110       24         23   1200000         0.001
111       24         12   1200000         0.001

[112 rows x 4 columns]


## Step 2: Visualize Graph

In [3]:
#Draw loaded graph
# spring layout prevents graph from
# becoming too clustered
layout = nx.spring_layout(G)
# draw node, edges, and labels sperately
nx.draw_networkx_labels(G, pos=layout )
nx.draw_networkx_nodes(G, pos=layout) 
nx.draw_networkx_edges(G, pos=layout)
# only add capacities as labels
edge_labels = dict([((source, dest), G[source][dest]["capacity"]) for source, dest in G.edges])
print(edge_labels)
nx.draw_networkx_edge_labels(G, pos= layout, edge_labels= edge_labels,font_color="red", font_weight="bold", font_size = 5)

{(2, 9): 1200000, (2, 3): 1200000, (2, 6): 1200000, (2, 17): 1200000, (2, 16): 1200000, (2, 25): 1200000, (2, 21): 1200000, (2, 15): 1200000, (2, 20): 1200000, (9, 22): 1200000, (9, 8): 1200000, (9, 13): 1200000, (9, 3): 1200000, (9, 16): 1200000, (9, 5): 1200000, (22, 13): 1200000, (22, 17): 1200000, (22, 23): 1200000, (22, 24): 1200000, (22, 21): 1200000, (8, 13): 1200000, (8, 3): 1200000, (8, 5): 1200000, (13, 5): 1200000, (13, 11): 1200000, (13, 17): 1200000, (13, 12): 1200000, (13, 16): 1200000, (13, 10): 1200000, (13, 15): 1200000, (3, 6): 1200000, (16, 17): 1200000, (16, 15): 1200000, (5, 7): 1200000, (5, 4): 1200000, (5, 14): 1200000, (6, 1): 1200000, (6, 25): 1200000, (6, 7): 1200000, (17, 19): 1200000, (17, 18): 1200000, (17, 21): 1200000, (17, 15): 1200000, (17, 20): 1200000, (25, 1): 1200000, (25, 7): 1200000, (21, 18): 1200000, (21, 15): 1200000, (20, 19): 1200000, (7, 4): 1200000, (14, 11): 1200000, (14, 10): 1200000, (11, 12): 1200000, (11, 10): 1200000, (12, 24): 120000

{(2, 3): Text(0.09514296095498673, 0.20684169766721985, '1200000'),
 (2, 6): Text(0.07665784907929996, 0.38721665026513585, '1200000'),
 (2, 9): Text(-0.008846291107174414, 0.03329668015456838, '1200000'),
 (2, 15): Text(-0.2213377358371479, -0.06801543539386887, '1200000'),
 (2, 16): Text(-0.14188333654796564, -0.05163819224818561, '1200000'),
 (2, 17): Text(-0.2912111927579902, -0.012419845985342057, '1200000'),
 (2, 20): Text(-0.40634937346966726, 0.13707212511532524, '1200000'),
 (2, 21): Text(-0.33913518007914056, 0.022039842599855364, '1200000'),
 (2, 25): Text(0.00048140445881939, 0.45913020925699577, '1200000'),
 (3, 6): Text(0.31691536252771835, 0.43167283055076233, '1200000'),
 (5, 4): Text(0.04116821967849887, 0.30639802257540283, '1200000'),
 (5, 7): Text(0.1634728116600775, 0.3088282214578858, '1200000'),
 (5, 14): Text(0.2729217172949009, -0.23443009231992512, '1200000'),
 (6, 1): Text(0.37122135005052603, 0.8060238915743392, '1200000'),
 (6, 7): Text(0.22244797990508447,

## Step 3: Parse Traffic Demands

In [4]:
demands = np.loadtxt("data/ATT/demand.txt")
demands_dict = {}
n = len(G.nodes())
i = 0
for t in demands:
    src = 0
    dest = 0
    for y in t:
        demands_dict[(i,src,dest)] = y
        if dest == n -1:
            dest = 0
            src += 1
        else:
            dest +=1
    i+=1
demands_dict

{(0, 0, 0): 6543.408364,
 (0, 0, 1): 2593.426585,
 (0, 0, 2): 13353.45958,
 (0, 0, 3): 3753.163709,
 (0, 0, 4): 1636.843883,
 (0, 0, 5): 7074.178405,
 (0, 0, 6): 5327.500337,
 (0, 0, 7): 11399.420783,
 (0, 0, 8): 1408.264286,
 (0, 0, 9): 4934.96452,
 (0, 0, 10): 15741.801504,
 (0, 0, 11): 49688.994885,
 (0, 0, 12): 747.073868,
 (0, 0, 13): 29622.790672,
 (0, 0, 14): 5373.333487,
 (0, 0, 15): 16550.998545,
 (0, 0, 16): 2028.071557,
 (0, 0, 17): 6088.208035,
 (0, 0, 18): 1615.715696,
 (0, 0, 19): 11637.764782,
 (0, 0, 20): 8755.448494,
 (0, 0, 21): 2479.975185,
 (0, 0, 22): 982.635976,
 (0, 0, 23): 33341.804091,
 (0, 0, 24): 9053.148028,
 (0, 1, 0): 1574.251068,
 (0, 1, 1): 957.789644,
 (0, 1, 2): 4275.116025,
 (0, 1, 3): 1118.74365,
 (0, 1, 4): 524.036297,
 (0, 1, 5): 2108.672253,
 (0, 1, 6): 1588.022169,
 (0, 1, 7): 3397.941207,
 (0, 1, 8): 419.775648,
 (0, 1, 9): 1471.015029,
 (0, 1, 10): 5039.744754,
 (0, 1, 11): 15907.953817,
 (0, 1, 12): 239.176031,
 (0, 1, 13): 9483.749612,
 (0, 1

## Step 4: Traffic Algorithm: Maximize Total Throughput


In [5]:
#TODO
#Decision variables:

#edge current flows and capacities
flow_matrix = {}
capacity_matrix = {}
for key, value in edge_labels:
    flow_matrix[key] = 0
    capacity_matrix[key] = value



# edge_time, demand = demands_dict


#If flow is 0, edge is not active

#for each edge in edges: set in some array/matrix all the possible edges

#----------------------------

#Optimization function:

#Sum of the flows of all of the edges

model = gb.Model()
print(edge_labels)

edges, edge2, capacities = edge_labels
print(edges)

flow = model.addVars(edges, obj=capacities, name="flow")

model.setObjective(gb.quicksum(flow_matrix.values()), gb.GRB_MAXIMIZE)


#----------------------------

#Constraints: 

#Capacity check: Flow on edge (u, v) <= capacity (u, v)

#Demand reqs: flow into node u = demand u: f_in(u) - f_out(u) = demand(u) https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/flowext.pdf

#Conservation: Flow into node u = flow out of node u???

# model.addConstr(flow_matrix[(t, u, v)] <= capacity_matrix[t, u, v]])
# model.addConstr(flow_in[t, u] - flow_out[t, u])

Restricted license - for non-production use only - expires 2024-10-28
{(2, 9): 1200000, (2, 3): 1200000, (2, 6): 1200000, (2, 17): 1200000, (2, 16): 1200000, (2, 25): 1200000, (2, 21): 1200000, (2, 15): 1200000, (2, 20): 1200000, (9, 22): 1200000, (9, 8): 1200000, (9, 13): 1200000, (9, 3): 1200000, (9, 16): 1200000, (9, 5): 1200000, (22, 13): 1200000, (22, 17): 1200000, (22, 23): 1200000, (22, 24): 1200000, (22, 21): 1200000, (8, 13): 1200000, (8, 3): 1200000, (8, 5): 1200000, (13, 5): 1200000, (13, 11): 1200000, (13, 17): 1200000, (13, 12): 1200000, (13, 16): 1200000, (13, 10): 1200000, (13, 15): 1200000, (3, 6): 1200000, (16, 17): 1200000, (16, 15): 1200000, (5, 7): 1200000, (5, 4): 1200000, (5, 14): 1200000, (6, 1): 1200000, (6, 25): 1200000, (6, 7): 1200000, (17, 19): 1200000, (17, 18): 1200000, (17, 21): 1200000, (17, 15): 1200000, (17, 20): 1200000, (25, 1): 1200000, (25, 7): 1200000, (21, 18): 1200000, (21, 15): 1200000, (20, 19): 1200000, (7, 4): 1200000, (14, 11): 1200000, (14

ValueError: too many values to unpack (expected 3)

In [ ]:
demand ={}
for (i, s, d) in demands_dict:
    demand[(s, d)] = demands_dict[(i, s, d)]
    
m = gp.Model('step4')
traffic = {}
for (i, j) in edge_labels:
    traffic[(i, j)] = m.addVar(ub=edge_labels[(i, j)], name=f"traffic_{i}_{j}")

for (i, j) in edge_labels:
    m.addConstr(gp.quicksum(traffic[(i, j)] - traffic[(j , i)], (i, j) in edge_labels) == demand[(i, j)])
for (i, j) in edge_labels:
    m.addConstr(traffic[(i, j)] <= edge_labels[(i, j)])
    
m.setObjective(gp.quicksum(traffic[(i, j)] for (i, j) in edge_labels), GRB.MAXIMIZE)

m.opi

## Step 5: Traffic Algorithm: Maximize Link Utilization

In [ ]:
demand = {}
for (i , s, d) in demands_dict:
    demand[(s, d)] = demands_dict[(i, s, d)]
model2 = gb.Model('step5')
x = {}
for u, v in edge_labels:
    x[u, v] = model2.addVar(vtype=GRB.CONTINUOUS)



## Step 6: Compare Algorithms

TODO

## Step 7 (EXTRA CREDIT): Scaling with Topology Size